In [1]:
!pip install gradio

In [2]:
import os
os.listdir()  # Check if the model file is present


['Website_for_Breast_Tumor_Detection_[1].ipynb']

In [10]:
import torch

model_path = "C:/Users/saran/Downloads/mipmini/deeplabv3_breast_ultrasound (2).pth"

try:
    checkpoint = torch.load(model_path, map_location="cpu")
    print("✅ Model file is valid!")
except Exception as e:
    print("❌ Model file is corrupted or invalid:", e)


✅ Model file is valid!


In [12]:
import torch

model_path = "C:/Users/saran/Downloads/mipmini/deeplabv3_breast_ultrasound (2).pth"

# Load DeepLabV3+ model
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=False)
model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1,1))  # Adjust output classes

# Load state_dict and remove aux_classifier keys
checkpoint = torch.load(model_path, map_location="cpu")

filtered_state_dict = {k: v for k, v in checkpoint.items() if "aux_classifier" not in k}

# Load the modified state_dict
model.load_state_dict(filtered_state_dict, strict=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("✅ Model loaded successfully without aux_classifier!")


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to C:\Users\saran/.cache\torch\hub\v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\saran/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:06<00:00, 26.0MB/s] 


✅ Model loaded successfully without aux_classifier!


In [ ]:
import torchvision.transforms as transforms
def segment_tumor(image):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)["out"]

    pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

    return pred_mask


In [27]:
import gradio as gr
import numpy as np


def process_image(image):
    pred_mask = segment_tumor(image)
    return np.uint8(pred_mask * 255)  # Convert to grayscale

gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="numpy"),
    title="Breast Tumor Segmentation",
    description="Upload a breast ultrasound image, and the model will segment the tumor region.",
    allow_flagging="never"
).launch(debug=True)


c:\Users\saran\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\saran\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\saran\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\saran\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\saran\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
      

Keyboard interruption in main thread... closing server.
